In [ ]:
from sumy.parsers.plaintext import PlaintextParser #We're choosing a plaintext parser here, other parsers available for HTML etc.
from sumy.nlp.tokenizers import Tokenizer 
from sumy.summarizers.lex_rank import LexRankSummarizer #We're choosing Lexrank, other algorithms are also built in
import pandas as pd, nltk
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
df = pd.read_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/data_train.csv', encoding='utf-8')

In [ ]:
stopword_list = nltk.corpus.stopwords.words('german')
def remove_stopwords(text):
    filtered_words = [word for word in nltk.word_tokenize(text) if word not in stopword_list]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

def lexRank(df, size):
    lst = []
    parser = PlaintextParser.from_string(df, Tokenizer("german"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, size)
    for s in summarizer(parser.document, size):
        lst.append(s)
    string = ' '.join([str(e) for e in lst])
    return string

In [ ]:
test_df = df.head(50)
print(lexRank(test_df['source'][0], 1))

In [ ]:
test_df['source'] = test_df['source'].apply(remove_stopwords)
test_df['lex1'] = test_df.apply(lambda x: lexRank(x['source'], 1), axis=1)
test_df['lex2'] = test_df.apply(lambda x: lexRank(x['source'], 2), axis=1)
test_df['lex5'] = test_df.apply(lambda x: lexRank(x['source'], 5), axis=1)
test_df['lex10'] = test_df.apply(lambda x: lexRank(x['source'], 10), axis=1)
test_df.to_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/lextR2.csv')

In [ ]:
df = pd.read_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/lextR2.csv', encoding='utf-8')

In [ ]:
test_df.head()

In [ ]:
import re
def cl_output(df):
    return re.sub(r'\(\<\b\Sentence\b: |(\>,)|(>\))|( \<\b\Sentence\b: )','' ,df)

test_df['lex1'] = test_df.apply(lambda x: cl_output(str(x['lex1'])), axis=1)
test_df['lex2'] = test_df.apply(lambda x: cl_output(str(x['lex2'])), axis=1)
test_df['lex5'] = test_df.apply(lambda x: cl_output(str(x['lex5'])), axis=1)
test_df['lex10'] = test_df.apply(lambda x: cl_output(str(x['lex10'])), axis=1)
test_df.to_csv('G:/Extractive-Summarisation-of-German-Wikipedia/dataset/lextR2.csv')

In [ ]:
from rouge import Rouge
import sys

sys.stdout = open("G:/Extractive-Summarisation-of-German-Wikipedia/dataset/test.txt", "w")

hy, rf = test_df['lex1'], test_df['summary']
rouge= Rouge()
scoresT1= rouge.get_scores(hy, rf, avg=True)
print("LextRank1 scores: ", scoresT1, '\n\n')

hy2 = test_df['lex2']
scoresT2= rouge.get_scores(hy2, rf, avg=True)
print("LextRank2 scores: ", scoresT2, '\n\n')

hy5 = test_df['lex5']
scoresT5= rouge.get_scores(hy5, rf, avg=True)
print("LextRank5 scores: ", scoresT5, '\n\n')

hy10 = test_df['lex10']
scoresT10= rouge.get_scores(hy10, rf, avg=True)
print("LextRank10 scores: ", scoresT10, '\n\n')

sys.stdout.close()